In [20]:
import os
import random
import numpy as np
import pandas as pd
import collections
import math

#things associated with a label 

1- counter 
2- probability 
3- function for individual probability 


And so when you create a class, you're not defining an object you're defining like something that that tells the operating length or the language that you're working in, what objects of this type look like. So for us objects of this type have a counter objects of this type have a probability and object of this type have a certain handful of functions associated with them. And, and that's what we'll go sort of capture in this in this in this code.

In [21]:
class LabelClass: 
    def __init__(self, label):
        self.counter= collections.Counter()
        self.label= label 

    def calc_probs(self, alpha):
        self.probs = {key: (alpha + value) / (sum(self.counter.values()) + (alpha * len(self.counter))) for key, value in self.counter.items()}

    def calc_prob_sen(self, sentence, label_prob):
        return math.prod([self.probs.get(word,0) for word in sentence])*label_prob[self.label]



In [22]:
uri_train  = 'https://raw.githubusercontent.com/thiagorainmaker77/liar_dataset/master/train.tsv'

df_train = pd.read_table(uri_train,
                             names = ['id',	'label'	,'statement',	'subject',	'speaker', 	'job', 	'state',	'party',	'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c',	'venue'])


labels= ['true', "false", "half-true", "barely-true"]
other_df = pd.DataFrame({"label": labels})

df_linear = df_train[df_train.label.isin(other_df.label)]

df_linear_1= df_linear.reset_index()
df_linear_1['statement'] = df_linear_1['statement'].str.lower()
#df_linear_1['statement']= df_linear_1['statement'].str.split()

label_dict= {}

for i in labels: 

    label_dict[i]= LabelClass(i)

#print(label_dict)



In [23]:
uri_test  = 'https://raw.githubusercontent.com/thiagorainmaker77/liar_dataset/master/test.tsv'

df_test = pd.read_table(uri_train,
                             names = ['id',	'label'	,'statement',	'subject',	'speaker', 	'job', 	'state',	'party',	'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c',	'venue'])

other_df_1 = pd.DataFrame({"label": ['true', "false", "half-true", "barely-true"]})

df_linear_test = df_test[df_test.label.isin(other_df_1.label)]

df_linear_test_1= df_linear_test.reset_index()
df_linear_test_1['statement'] = df_linear_test_1['statement'].str.lower()
#df_linear_1['statement']= df_linear_1['statement'].str.split()


In [24]:
def naive_bayes_alpha(train_df, test_df, alpha: int): 

    df_linear_1 = train_df
    df_linear_test_1 = test_df

    ###################################################################
    label_cnt = collections.Counter(df_linear_1.label)
    # true_cnt = collections.Counter() #true_cnt is p(x|true)
    # false_cnt = collections.Counter()
    # half_true_cnt= collections.Counter()
    # barely_true_cnt= collections.Counter()



    for idx, row in df_linear_1.iterrows():

        counter= label_dict[row["label"]].counter
        counter.update(row["statement"].split())


    ################################################################### 
    label_probs = {key: value / sum(label_cnt.values()) for key, value in label_cnt.items()}

    for key, value in label_dict.items():
        value.calc_probs(alpha)

    ##############L####################################################

    to_calc =[]

    for idx, row in df_linear_test_1.iterrows(): 

        sentence = row["statement"].split() 
        prob_arr = []
        for label in labels:
            label_obj = label_dict[label]
            prob_arr.append(label_obj.calc_prob_sen(sentence, label_probs))
        # probability_matrix = np.array([label_arr, prob_arr])
        max_prob_index = np.argmax(prob_arr)
        max_prob_label = labels[max_prob_index]
        curr_probs = max_prob_label


        #this is to calc accurracy later 
        if curr_probs == row["label"]: 
            to_calc.append(1)
        else: 
            to_calc.append(0)

    ###########################################################################

    count_correct = to_calc.count(1)
    rows_count = df_linear_test_1.shape[0]
    accuracy = count_correct/rows_count



    return(accuracy) 
    

        

In [25]:
naive_bayes_alpha(df_linear_1, df_linear_test_1, 0.5)

0.9154456244118833

In [9]:
empt_list= []

for i in range(100): 

    curr_accurange= naive_bayes_alpha(df_linear_1, df_linear_test_1, i)

    curr_list= [i,curr_accurange]

    empt_list.append(curr_list)



In [ ]:
#smoothing has not helped. the accuracy keeps on dropping 

